In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [2]:
browser = webdriver.Chrome()
url='https://www.billboard.com/charts/hot-100/2020-10-17/'
browser.get(url)
time.sleep(1)

In [3]:
result=[]

In [7]:
for i in range(11, 47):
    #크게 지정
    browser.maximize_window()
    #스크롤 내림
    browser.execute_script("window.scrollBy(0, 330);")
    time.sleep(2)
    
    #Billboard에서 날짜를 선택하는 icon을 찾는다
    calendar_list=browser.find_elements('css selector','main > div > div > div > div > div > div > div > a > svg')
    calendar_list[1].click()
    time.sleep(1)
    
    #날짜를 클릭하면 다음달로 넘어가는 화살표가 보인다.
    down_list=browser.find_elements('css selector','div > button.e-next')
    down_list[0].click()
    time.sleep(1)
    
    #일주일간 변화는 차이가 없어 한달간 변화를 보려고
    #똑같은 주에 대한 정보를 클릭
    date_list=browser.find_elements('css selector','div > table > tbody > tr')
    date_list[3].click()
    time.sleep(40)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    tag_list=soup.select('div.o-chart-results-list-row-container')
    
    #2020년 11월부터 2023년 10월까지 크롤링하려고 했는데
    #날짜를 잘못 지정해서 데이터를 합칠때 수정하였음
    month=i%12
    if(month==0):
        month=12
        up=(i//12)-1
        year=2019+up
    else:
        up=i//12
        year=2019+up
    
    for tag in tag_list:
        title=tag.select('li > #title-of-a-story')[0].text.strip()
        singer=tag.select('li.lrv-u-width-100p > ul > li > span')[0].text.strip()
        data=year,month,title,singer
        result.append(data)

In [8]:
import pandas as pd
df = pd.DataFrame(result)
df.columns =['년도','월','제목','가수']

In [10]:
data=[]
for i in tqdm(range(len(df))):
    rank=(i+1)%100
    if rank == 0:
        rank = 100
    data.append(rank)

100%|██████████| 3600/3600 [00:00<00:00, 716975.04it/s]


In [11]:
df['순위']=data

In [15]:
df=df[['순위','년도','월','제목','가수']]

In [14]:
df.to_excel('./빌보드차트.xlsx', index = False)